In [1]:
from gurobipy import Model, GRB, quicksum

In [2]:
# Parameters
m = 6  # Sutun sayısı
n = 16  # Satır sayısı
w = 10  # Dalga sayısı
v = 12  # Dalgadaki araç sayısı

In [3]:
# Sets
M = range(1, m + 1)  # Sutun Seti
N = range(1, n + 1)  # Satır Seti
K = range(1, w + 1)  # Dalga Seti
L = range(1, v + 1)  # Araç Seti

In [4]:
# Create the Gurobi model
model = Model("AMPL_Model")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-05


In [5]:
# Variables
x = model.addVars(M, N, K, L, vtype=GRB.BINARY, name="x")
y = model.addVars(K, vtype=GRB.BINARY, name="y")
z = model.addVars(M, N, K, vtype=GRB.BINARY, name="z")
r = model.addVars(M, N, vtype=GRB.CONTINUOUS, lb=0, name="r")

In [6]:
# Objective: Maximize the number of waves
model.setObjective(quicksum(y[k] for k in K), GRB.MAXIMIZE)

In [8]:
# Constraints
# Constraint 1: Each cell can have at most one vehicle
model.addConstrs((quicksum(x[i, j, k, l] for l in L for k in K) <= 1 for i in M for j in N), "Const1")

# Constraint 2: Each vehicle can occupy at most one position
model.addConstrs((quicksum(x[i, j, k, l] for i in M for j in N) <= 1 for k in K for l in L), "Const12")

# Constraint 3: A wave is loaded if all its vehicles are loaded
model.addConstrs((quicksum(x[i, j, k, l] for i in M for j in N) >= y[k] for k in K for l in L), "Const2")

# Constraint 4: Waves should load sequentially
model.addConstrs((y[k] >= y[k + 1] for k in range(1, w)), "Const22")

# Constraint 5: First vehicle of the first wave must be placed in the first cell
model.addConstr((quicksum(x[i, 1, 1, 1] for i in M) == 1), "Const3")

# Constraint 6: Ensure order of vehicles in subsequent waves
model.addConstrs(
    (quicksum(x[i, j, k, l] for l in L) +
     (1 / (j - 1)) * quicksum(x[i, a, b, l] for l in L for a in range(1, j) for b in range(k + 1, w + 1)) <= 1
     for i in M for j in range(2, n + 1) for k in range(1, w)),
    "Const4"
)

# Constraint 7: Balancing constraints for vehicle placement
model.addConstrs(
    (quicksum(x[i, j, a, l] * ((m / 2) + 1 - i) for i in range(1, int(m / 2) + 1) for j in N for a in range(k, w + 1) for l in L) -
     quicksum(x[i, j, a, l] * (i - (m / 2)) for i in range(int(m / 2) + 1, m + 1) for j in N for a in range(k, w + 1) for l in L) <= 2
     for k in K),
    "Const5"
)

model.addConstrs(
    (quicksum(x[i, j, a, l] * ((m / 2) + 1 - i) for i in range(1, int(m / 2) + 1) for j in N for a in range(k, w + 1) for l in L) -
     quicksum(x[i, j, a, l] * (i - (m / 2)) for i in range(int(m / 2) + 1, m + 1) for j in N for a in range(k, w + 1) for l in L) >= -2
     for k in K),
    "Const6"
)

# Constraint 8: Define z as a sum over x
model.addConstrs((quicksum(x[i, j, k, l] for l in L) == z[i, j, k] for i in M for j in N for k in K), "Const7")

# Constraint 9: Define r as a weighted sum of z
model.addConstrs((quicksum(k * z[i, j, k] for k in K) == r[i, j] for i in M for j in N), "Const8")

# Solve the model
model.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4464 rows, 12586 columns and 1146960 nonzeros
Model fingerprint: 0x756dc9b3
Variable types: 96 continuous, 12490 integer (12490 binary)
Coefficient statistics:
  Matrix range     [7e-02, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 3198 rows and 1007 columns
Presolve time: 1.45s
Presolved: 1266 rows, 11579 columns, 485858 nonzeros
Variable types: 0 continuous, 11579 integer (11559 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Use crossover to convert LP symmetric solution to basic solution..

In [9]:
# Display the values of r in a tabular format
if model.status == GRB.OPTIMAL:
    print("r [*,*] (tr)")
    print(":   ", end="")
    # Print the column headers (1 to n)
    for j in N:
        print(f"{j:4}", end="")
    print("   :=")
    
    # Print each row of r (1 to m)
    for i in M:
        print(f"{i:<4}", end="")  # Print the row header
        for j in N:
            # Print the value of r[i, j], formatted for alignment
            print(f"{r[i, j].X:4.0f}", end="")
        print()  # Newline at the end of the row
else:
    print("No optimal solution found.")


r [*,*] (tr)
:      1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16   :=
1      1   1   1   2   2   5   5   5   5   5   6   6   7   7   8   8
2      1   3   3   3   3   4   4   4   4   4   4   4   4   5   6   6
3      1   2   2   2   2   3   6   6   7   7   7   8   8   8   8   8
4      1   1   1   1   2   3   3   3   3   3   7   7   7   7   7   7
5      1   1   2   2   2   3   3   6   6   6   6   6   6   8   8   8
6      1   2   2   4   4   4   4   5   5   5   5   5   5   7   8   8
